# Imports and Installation

In [ ]:
!pip install groq
!pip install mediapipe
!pip install FER
# Download the shape_predictor_68_face_landmarks.dat file
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bunzip2 shape_predictor_68_face_landmarks.dat.bz2

In [ ]:
import os
from groq import Groq
import cv2
import random
import numpy as np
import dlib
from imutils import face_utils
from scipy.spatial import distance as dist
from fer import FER

# Initialize

In [38]:
groq_api_key = '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'
client = Groq(api_key = groq_api_key)

In [5]:
all_scores = {
    'content': {
        'total_questions': 0,
        'ans_validation_score': 0,
        'quality':{
            'unity': 0,
            'coherence': 0,
            'grammar' : 0,
            'optimal_repetetion' : 0,
            'clarity' : 0,
            'engagement': 0,
            'tone_and_style': 0,
        },
    'audio': {
        'fluency' : 0,
        'tone_and_clarity' : 0,
    },
    'video' : {
        'eye_contact' : 0,
        'expressions' : 0,
    }
    }
}

In [6]:
all_scores

{'content': {'total_questions': 0,
  'ans_validation_score': 0,
  'quality': {'unity': 0,
   'coherence': 0,
   'grammar': 0,
   'optimal_repetetion': 0,
   'clarity': 0,
   'engagement': 0,
   'tone_and_style': 0},
  'audio': {'fluency': 0, 'tone_and_clarity': 0},
  'video': {'eye_contact': 0, 'expressions': 0}}}

# Functions

In [41]:
def get_transcript_from_audio(filename):
    with open(filename, "rb") as file:
        translation = client.audio.translations.create(
          file=(filename, file.read()),
          model="whisper-large-v3",
          # prompt="Transcribe the following audio exactly as spoken, including all filler words, repetitions, stammering, and disfluencies. Do not correct grammar, smooth out phrases, or remove any hesitations. The transcription should faithfully represent the speaker's words as they were spoken.",  # Optional
          response_format="json",  # Optional
          temperature=0.0  # Optional
        )
    return translation.text

In [2]:
def fetch_audio_from_video(video_path):
    #goutam
    return 

In [44]:
def get_json_from_response(response):
    json = ''
    return json

## content

In [43]:
def get_conversation_from_audio(audio):
    # goutam 
    return 

In [34]:
def get_content_quality_scores(text_body):
    groq_input = get_groq_input(text_body)
    response_content_quality = get_groq_response(groq_input)
    json_response_content_quality = get_json_from_response(response_content_quality)
    return json_response_content_quality

In [16]:
def get_similiarity_score(context, text1, text2):
    # goutam
    return 

In [1]:
def get_content_ans_valid_score(qsn_ans_labelled):
    '''
    question_answers_with_ground_truth format --
    {
      '001': {
        'question':  ...,
        'ans_by_candidate' : ...,
        'ans_actual' :  ...,
        'score': ...,
    },
    ...
    }
    '''
    qsns = qsn_ans_labelled.keys() 
    for qsn in qsns:
        question = qsn_ans_labelled[qsn]['question']
        ans_by_candidate = qsn_ans_labelled[qsn]['ans_by_candidate']
        ans_actual = qsn_ans_labelled[qsn]['ans_actual']
        qsn_ans_labelled[qsn]['score'] = get_similiarity_score(question, ans_by_candidate, ans_actual)
    
    total_qsns = len(qsns)
    overall_score_ans_validation = [qsn_ans_labelled[qsn]['score'] for qsn in qsns]/total_qsns

    return total_qsns, overall_score_ans_validation

## audio

In [27]:
def get_audio_fluency_score(conversation):
    #tej
    # speech speed consistency

    # filler words - 'uh'
    return

In [28]:
def get_audio_tone_and_clarity_score(audio):
    #tej
    return 

## video

In [ ]:
# def get_expresion_for_image(image):
#     return dominant_expression, expression_dic

In [ ]:
def fetch_random_n_images(video_path, n):
    # fetch n random frames
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    random_frames = sorted(random.sample(range(total_frames), n))
    frame_number = 0
    while True:
        sucess, frame = cap.read()
        if not sucess:
            break
        frame_number+=1
        frame_name = str(frame_number).zfill(6)
        if frame_number in random_frames:
            cv2.imwrite(f'{output_folder}/{filename}/{frame_name}.jpg')


In [ ]:
def initialize_detectors():
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    fer_detector = FER()
    return detector, predictor, fer_detector

In [ ]:
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear


In [ ]:
def calculate_eyeball_movement(landmarks):
    left_eye = landmarks[42:48]
    right_eye = landmarks[36:42]
    left_eye_center = left_eye.mean(axis=0).astype("int")
    right_eye_center = right_eye.mean(axis=0).astype("int")
    left_ear = eye_aspect_ratio(left_eye)
    right_ear = eye_aspect_ratio(right_eye)
    ear = (left_ear + right_ear) / 2.0
    return left_eye_center, right_eye_center, ear


In [7]:
def get_video_analysis(video_path, save_video = False):
    # Initialize detectors
    detector, predictor, fer_detector = initialize_detectors()

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Get video details
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    if save_video:
        # Define the codec and create a VideoWriter object to save the output video
        output_path = video_path.split('.')[0] + '_output.mp4'
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Initialize variables
    movement_counts = []
    blink_counts = 0
    emotion_counts = {'angry': 0, 'disgust': 0, 'fear': 0, 'happy': 0, 'sad': 0, 'surprise': 0, 'neutral': 0}
    prev_left_eye_center = None
    prev_right_eye_center = None

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 0)

        for rect in rects:
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)
            left_eye_center, right_eye_center, ear = calculate_eyeball_movement(shape)

            if prev_left_eye_center is not None and prev_right_eye_center is not None:
                left_movement = dist.euclidean(prev_left_eye_center, left_eye_center)
                right_movement = dist.euclidean(prev_right_eye_center, right_eye_center)
                movement_counts.append(left_movement + right_movement)

                if ear < 0.21:  # Threshold for blinking
                    blink_counts += 1

                # Draw eye centers and EAR on the frame
                cv2.circle(frame, tuple(left_eye_center), 2, (0, 255, 0), -1)
                cv2.circle(frame, tuple(right_eye_center), 2, (0, 255, 0), -1)
                cv2.putText(frame, f"EAR: {ear:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            prev_left_eye_center = left_eye_center
            prev_right_eye_center = right_eye_center

        # Emotion detection
        result = fer_detector.detect_emotions(frame)
        for face in result:
            emotions = face['emotions']
            top_emotion = max(emotions, key=emotions.get)
            emotion_counts[top_emotion] += 1
            cv2.putText(frame, f"Emotion: {top_emotion}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

        # Calculate average movement and blink count
        avg_movement_rate = np.mean(movement_counts) if movement_counts else 0

        # Evaluate the expression-based description
        total_frames = sum(emotion_counts.values())
        emotion_percentages = {emotion: (count / total_frames) * 100 for emotion, count in emotion_counts.items()} if total_frames > 0 else {emotion: 0 for emotion in emotion_counts}

        dominant_emotion = max(emotion_percentages, key=emotion_percentages.get)
        dominant_emotion_percentage = emotion_percentages[dominant_emotion]

        # Generate detailed description
        description = ""
        confidence_score = 8  # Default confidence score

        if dominant_emotion in ['fear', 'disgust']:
            description = (
                f"The candidate appears quite uneasy with a dominant emotional state of {dominant_emotion} "
                f"at {dominant_emotion_percentage:.2f}%. Additionally, there is some evidence of nervousness "
                f"indicated by eye movements and blinking. Overall, the candidate's demeanor suggests a higher "
                f"level of anxiety."
            )
            confidence_score -= 2
        elif dominant_emotion in ['happy', 'neutral']:
            description = (
                f"The candidate seems relaxed or positive with a dominant emotional state of {dominant_emotion} "
                f"at {dominant_emotion_percentage:.2f}%. The eye movement and blinking are within normal ranges, "
                f"indicating that the candidate is calm and composed throughout the video."
            )
            confidence_score += 1
        else:
            description = (
                f"The candidate shows a mixed emotional state with no dominant emotion. Eye movement and blink rate "
                f"are moderate, suggesting that while the candidate might not be extremely anxious, there are "
                f"slight signs of nervousness. The overall demeanor is neutral."
            )

        if avg_movement_rate > 2 or blink_counts > 20:
            description += (
                " The eye movement and blink count suggest some level of nervousness, adding to the overall "
                "impression of anxiety or unease."
            )
            confidence_score -= 2

        # Ensure confidence score is within 0-10 range
        confidence_score = max(0, min(10, confidence_score))

        # Display calculated parameters
        cv2.putText(frame, f"Avg Movement Rate: {avg_movement_rate:.2f}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(frame, f"Blink Count: {blink_counts}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(frame, f"Dominant Emotion: {dominant_emotion.capitalize()} ({dominant_emotion_percentage:.2f}%)", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(frame, f"Overall Description: {description}", (10, 180), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(frame, f"Confidence Score: {confidence_score}/10", (10, 210), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # Write the annotated frame to the output video
        if save_video:
            out.write(frame)

    # Release the video capture and writer objects
    cap.release()
    out.release()

    # Output results
    print(f"Output video saved to {output_path}")
    print(f"Average Eye Movement Rate: {avg_movement_rate:.2f}")
    print(f"Blink Count: {blink_counts}")
    print(f"Emotion Counts: {emotion_counts}")
    print(f"Overall Description: {description}")
    print(f"Confidence Score: {confidence_score}/10")

    return avg_movement_rate, blink_counts, emotion_counts, confidence_score

In [3]:
all_scores

NameError: name 'all_scores' is not defined

In [ ]:
def fill_scores():
    all_scores['content']['quality'] = get_content_quality_scores()
    all_scores['content']['quality'] = 

In [12]:
video_path = ''
output_folder = 'output'
filename = video_path.split('/')[-1].split('.')[0]